## Service Mesh mit Fault-Injection

Dieses Hands-On führt dich Schritt für Schritt durch die Fehlerbehebung bei einem fehlerhaften Microservice in einer Istio-verwalteten Umgebung.

---

**Ziel**
- Identifiziere den fehlerhaften Microservice mithilfe von Istio Telemetry
- Konfiguriere eine Retry-Policy, um die Auswirkung vorübergehender Fehler zu minimieren.
- Optimiere Ressourcen oder Latenzzeiten des betroffenen Services.

### Namespace erstellen und Istio aktivieren

In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl label namespace ms-rest istio-injection=enabled


### Deployment der Microservices

In [ ]:
%%bash
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/catalog-pod.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/customer-pod.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/order-pod.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/webshop-pod.yaml

### Bereitstellung der Services

In [ ]:
%%bash
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/webshop-service.yaml


### Status der Deployments und Services überprüfen

In [ ]:
%%bash
kubectl get pod,services --namespace ms-rest

### Webshop-URL ermitteln

In [ ]:
%%bash
echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop


### Fault-Injection

Istio unterstützt Fault-Injection zur Simulation von Fehlern (z. B. Latenz oder HTTP-Fehlercodes)


In [ ]:
%%bash
cat <<EOF | kubectl apply -f - 
apiVersion: networking.istio.io/v1beta1
kind: VirtualService
metadata:
  name: customer-fault-injection
  namespace: ms-rest
spec:
  hosts:
  - customer.ms-rest.svc.cluster.local
  http:
  - match:
    - sourceLabels:
        app: order
    fault:
      delay:
        percentage:
          value: 50.0 # 50% der Anfragen verzögern
        fixedDelay: 5s # Verzögerung um 5 Sekunden
      abort:
        percentage:
          value: 20.0 # 20% der Anfragen abbrechen
        httpStatus: 500 # Mit HTTP 500 abbrechen
    route:
    - destination:
        host: customer
        port:
          number: 8080
EOF

### Lasttest durchführen

In [ ]:
%%bash
# LastTest mit 404
URL="http://localhost":$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop
hey -z 10s -c 50  ${URL}/order/order

Der Traffic sollte in Kiali, Jaeger oder Zipkin sichtbar sein.

### Aufräumen

In [ ]:
%%bash
kubectl delete namespace ms-rest

### Quellen
- [Sourcecode](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v2.1.0?ref_type=heads)
- [Kubernetes Deklarationen](https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates)
- [Container Registry](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/container_registry)